In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from datetime import date
today = date.today()
import re

In [4]:
# Read the Excel file into a pandas DataFrame
data = pd.read_excel('PALOMA_sensor.xlsx', header=0)

data.columns = [re.sub(r'[-%/ ]', '_', col) for col in data.columns]

for column in data.columns:
    # Convert the column to a NumPy array
    np_array = data[column].to_numpy()
    
    # Convert the data type to int32 or float32
    if np_array.dtype == 'int64':
        np_array = np_array.astype('int32')
    elif np_array.dtype == 'float64':
        np_array = np_array.astype('float32')
    
    # Delete the old NumPy array if it exists
    if column in locals():
        del locals()[column]
    
    # Use the modified column name as the variable name
    globals()[column] = np_array

In [5]:
whos

Variable           Type         Data/Info
-----------------------------------------
DO_corr_umol_kg    ndarray      19: 19 elems, type `float32`, 76 bytes
DO_raw_umol_kg     ndarray      19: 19 elems, type `float32`, 76 bytes
Date_Time          ndarray      19: 19 elems, type `object`, 152 bytes
Depth_dbar         ndarray      19: 19 elems, type `float32`, 76 bytes
Fixed_Stations     ndarray      19: 19 elems, type `object`, 152 bytes
Latitude           ndarray      19: 19 elems, type `float32`, 76 bytes
Longitude          ndarray      19: 19 elems, type `float32`, 76 bytes
S_SBE37_ODO        ndarray      19: 19 elems, type `float32`, 76 bytes
T_SBE37_ODO_degC   ndarray      19: 19 elems, type `float32`, 76 bytes
column             str          DO_corr_umol_kg
data               DataFrame       Fixed_Stations        <...>213.82        220.96254  
date               type         <class 'datetime.date'>
np                 module       <module 'numpy' from '/ho<...>kages/numpy/__init__.py

In [8]:
time=Date_Time
Lat=Latitude
Lon=Longitude

DO=DO_corr_umol_kg
Depth=Depth_dbar
Salinity=S_SBE37_ODO
pCO2= pCO2_corr_uatm
Temperature=T_SBE37_ODO_degC


In [9]:
ds=xr.Dataset(
    data_vars=dict(
        Oxygen=(["time"], DO,{"units": "umol/kg",
                                            "standard_name": "mole_concentration_of_dissolved_molecular_oxygen_in_sea_water",
                                            "short_name": "DO",
                                            "long_name": "mole_concentration_of_dissolved_molecular_oxygen_in_sea_water",
                                            "description": "Molarity of dissolved oxygen logged by SBE37-ODO (umol/kg)",
                                            "valid_min": "0",
                                            "valid_max": "400",
                                            "missing_value": "-9999999",
                                            "axis": "time",
                                            "history": ""}),
        
        Depth=(["time"], Depth,{"units": "dbar",
                                            "standard_name": "depth",
                                            "short_name": "depth",
                                            "long_name": "depth",
                                            "description": "depth (pressure) of sensor",
                                            "valid_min": "0",
                                            "valid_max": "10000",
                                            "missing_value": "-9999999",
                                            "axis": "time",
                                            "history": ""}),
        

        Salinity_ODO=(["time"], Salinity,{"units": "psu",
                                            "standard_name": "sea_water_salinity",
                                            "short_name": "so",
                                            "long_name": "Sea Water Salinity",
                                            "description": "Sea water salinity is the salt content of sea water on the Practical Salinity Scale, measured by SBE37-ODO",
                                            "valid_min": "0",
                                            "valid_max": "100",
                                            "missing_value": "-9999999",
                                            "axis": "time",
                                            "history": ""}),

        
        pCO2=(["time"], pCO2,{"units": "uatm",
                                            "standard_name": "surface_partial_pressure_of_carbon_dioxide_in_sea_water",
                                            "short_name": "pCO2",
                                            "long_name": "Surface partial pressure of carbon dioxide in sea water ",
                                            "description": " pCO2 in water, from Contros HydroC, corrected (uatm)",
                                            "valid_min": "0",
                                            "valid_max": "1000",
                                            "missing_value": "-9999999",
                                            "axis": "time",
                                            "history": ""}),
    
        
        Temperature_ODO=(["time"], Temperature,{"units": "degree Celcius",
                                            "standard_name": "sea_water_temperature",
                                            "short_name": "SW_Temperature",
                                            "long_name": "Sea Water Temperature",
                                            "description": "Sea water temperature is the in situ temperature of the sea water measured by the SBE37-ODO sensor",
                                            "valid_min": "-10",
                                            "valid_max": "100",
                                            "missing_value": "-9999999",
                                            "axis": "time",
                                            "history": ""})
        
       
    ),
    coords=dict(
    lon=(["lon"], Lon,{"units": "degrees East",
                                        "standard_name": "longitude",
                                        "short_name": "lon",
                                        "long_name": "Longitude",
                                        "description": "The Longitude represents the degrees east",
                                        "valid_min": "0",
                                        "valid_max": "360",
                                        "missing_value": "-9999999",
                                        "history": ""}),
                       
    lat=(["lat"], Lat,{"units": "degrees North",
                                        "standard_name": "latitude",
                                        "short_name": "lat",
                                        "long_name": "Latitude",
                                        "description": "The latitude represents the degrees north",
                                        "valid_min": "-90",
                                        "valid_max": "90",
                                        "missing_value": "-9999999",
                                        "history": ""}),
    time=time,
    reference_time=time[0],
)
)





In [10]:
ds.attrs['id']='DOI needed'

ds.attrs['naming_authority']='Istituto Nazionale di Oceanografia e Geofisica Sperimentale (OGS), Trieste, Italy '

ds.attrs['title'] = "Paloma_sensor"

ds.attrs['summary']='Dataset containing the SBE37-ODO sensor measuremets from the PALOMA station. Dissolved oxygen (DO) data measured by the sensor is corrected (DO_corr) using discrete oxygen samples analysed by winkler titration.'
ds.attrs['keywords']='Hydrography'
ds.attrs['keywords_vocabulary']='Temperature, Salinity, DO, pH, PO, SI, Alk'

ds.attrs['geospatial_lat_min']=np.min(Lat)
ds.attrs['geospatial_lat_max']=np.max(Lat)
ds.attrs['geospatial_lon_miN']=np.min(Lon)
ds.attrs['geospatial_lon_max']=np.max(Lon)

ds.attrs['time_coverage_start']=str(np.min(time))
ds.attrs['time_coverage_end']=str(np.max(time))

ds.attrs['Conventions']='CF 1.11'

ds.attrs['processing_level']='LEVEL 2'

ds.attrs['date_created']=str(today)

ds.attrs['creator_type']='Group'
ds.attrs['creator_institution']='Geophysical Institute'
ds.attrs['creator_name']='Joan Mateu Horrach Pou'
ds.attrs['creator_email']='joan.pou@uib.no'
ds.attrs['creator_url']=''

ds.attrs['institution']='Geophysical Institute'

ds.attrs['publisher_name']='Joan Mateu Horrach Pou'
ds.attrs['publisher_email']='joan.pou@uib.no'
ds.attrs['publisher_url']='I need the url'

ds.attrs['project']='ICOS'

ds.attrs['platform']='PALOMA'
ds.attrs['platform_vocabulary']='PALOMA_sensor'

ds.attrs['instrument']='SBE37-ODO, Contros HydroC CO2 sensor, water samples for winkler correction'


ds.attrs['iso_topic_category']='Needed'
ds.attrs['activity_type']='in situ observation'
ds.attrs['operational_status']='active'


In [11]:
comp = dict(zlib=True, complevel=5)
encoding = {var: comp for var in ds.data_vars}
ds.to_netcdf('Paloma_sensor.nc', encoding=encoding)
